<a href="https://colab.research.google.com/github/JuanPabl07DP/Cardiovascular_FNN/blob/main/Cardiovarcular_FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cardiovascular-disease-dataset


In [10]:
import pandas as pd

df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [11]:
import numpy as np

# Eliminar columnas innecesarias (ID si existe)
df = df.drop(columns=['id'], errors='ignore')

# Convertir 'gender' a binario (0 = mujer, 1 = hombre)
df['gender'] = df['gender'].map({1: 1, 2: 0})

# Revisar valores extremos
df = df[(df['height'] > 100) & (df['height'] < 250)]
df = df[(df['weight'] > 30) & (df['weight'] < 200)]
df = df[(df['ap_hi'] > 70) & (df['ap_hi'] < 250)]
df = df[(df['ap_lo'] > 40) & (df['ap_lo'] < 200)]

# Normalización
from sklearn.preprocessing import MinMaxScaler

features = df.drop(columns=['cardio'])
labels = df['cardio']

scaler = MinMaxScaler()
X = features.values
y = labels.values

In [12]:
from sklearn.model_selection import train_test_split

# División
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_dev, y_train, y_dev = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

# Escalado
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

print(f'Train: {len(X_train)} samples')
print(f'Dev: {len(X_dev)} samples')
print(f'Test: {len(X_test)} samples')

Train: 48119 samples
Dev: 10279 samples
Test: 10306 samples


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(256, input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.4),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_dev_scaled, y_dev),
                    epochs=50,
                    batch_size=64,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


752/752 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.7104 - loss: 0.7905 - val_accuracy: 0.7276 - val_loss: 0.6450 - learning_rate: 5.0000e-04
Epoch 2/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7306 - loss: 0.6268 - val_accuracy: 0.7257 - val_loss: 0.5938 - learning_rate: 5.0000e-04
Epoch 3/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7292 - loss: 0.5885 - val_accuracy: 0.7274 - val_loss: 0.5718 - learning_rate: 5.0000e-04
Epoch 4/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7364 - loss: 0.5674 - val_accuracy: 0.7274 - val_loss: 0.5639 - learning_rate: 5.0000e-04
Epoch 5/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7326 - loss: 0.5621 - val_accuracy: 0.7315 - val_loss: 0.5608 - learning_rate: 5.0000e-04
Epoch 6/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7335 - loss: 0.5561 - val_accuracy: 0.7303 - val_loss: 0.5571 - learning_rate: 5.0000e-04
Epoch 7/50
752/752 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7348 - lo

In [15]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7303 - loss: 0.5476
Test Accuracy: 0.7296, Test Loss: 0.5504


In [ ]:
model.save("modelo_cardio_hibrido_fnn.h5")